In [174]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

IterativeImputer_seed=0
adc_features = []
labels_path  = "/data1/practical-sose23/morphometric/picai_labels_cropped"
table_path   = labels_path+"/clinical_information/marksheet.csv"
table_path_filled =labels_path+"/clinical_information/marksheet_filled_normalized.csv"

t2w_radiomics  = pd.read_pickle("/data1/practical-sose23/morphometric/t2w_radiomic_features.pkl")
adc_radiomics  = pd.read_pickle("/data1/practical-sose23/morphometric/adc_radiomic_features.pkl")
clincical_info = pd.read_csv(table_path)

In [175]:
t2w_radiomics.shape,adc_radiomics.shape,clincical_info.shape

((1500, 143), (1500, 143), (1498, 12))

In [176]:
t2w_radiomics  = t2w_radiomics.set_index("study_id") 
adc_radiomics  = adc_radiomics.set_index("study_id")

In [177]:
clincical_info["case_csPCa"] =clincical_info["case_csPCa"].replace({'YES': 1, 'NO': 0})
clincical_info_toDrop=["mri_date","histopath_type","lesion_GS","lesion_ISUP"]
print("Dropping {} from clincical_info ".format(len(clincical_info_toDrop)))

Dropping 4 from clincical_info 


In [178]:
t2w_adc_feats =  t2w_radiomics.join(adc_radiomics, on="study_id",lsuffix='_T2W',rsuffix='_ADC')
full_feats =  clincical_info.join(t2w_adc_feats, on="study_id", how="inner").drop(columns=clincical_info_toDrop)

In [179]:
print("t2w_adc_feats={},full_feats={}".format(t2w_adc_feats.shape,full_feats.shape))
t2w_adc_feats_toDrop = full_feats.filter(like="diagnostics_").columns
print("Dropping {} diag t2w and adc feats from full_feats".format(len(t2w_adc_feats_toDrop)))
full_feats = full_feats.drop(columns=t2w_adc_feats_toDrop)
t2w_adc_feats = t2w_adc_feats.drop(columns=t2w_adc_feats_toDrop)
print("t2w_adc_feats={},full_feats={}".format(t2w_adc_feats.shape,full_feats.shape))

t2w_adc_feats=(1500, 284),full_feats=(1498, 292)
Dropping 74 diag t2w and adc feats from full_feats
t2w_adc_feats=(1500, 210),full_feats=(1498, 218)


## Showing NUll or NaN and their count

In [120]:
full_feats[full_feats.isna().any(axis=1)]

,patient_id,study_id,patient_age,psa,psad,prostate_volume,case_ISUP,case_csPCa,original_shape_Elongation_T2W,original_shape_Flatness_T2W,...,original_gldm_LargeDependenceEmphasis_ADC,original_gldm_LargeDependenceHighGrayLevelEmphasis_ADC,original_gldm_LargeDependenceLowGrayLevelEmphasis_ADC,original_gldm_LowGrayLevelEmphasis_ADC,original_gldm_SmallDependenceEmphasis_ADC,original_gldm_SmallDependenceHighGrayLevelEmphasis_ADC,original_gldm_SmallDependenceLowGrayLevelEmphasis_ADC,ellipse_vol_ADC,patient_id_ADC,prostate_volume_ADC
0,10000,1000000,73,7.7,NaN,55.0,0,0,0.765154,0.635129,...,662.6213506858952,2582.7996834329933,182.57676749912065,0.28133518730214563,0.0017020923999421496,0.006513236251134446,0.0004993064371440754,29.154570,10000,55.0
3,10003,1000003,72,13.0,NaN,71.5,0,0,0.97144,0.795884,...,691.217092990951,2764.41773495793,172.9169324992062,0.2504651035878711,0.0015755434930430682,0.006258803746919392,0.00040472842957398715,72.334082,10003,71.5
9,10009,1000009,65,24.0,NaN,120.0,0,0,0.953044,0.79555,...,696.0636754490004,2784.254659437479,174.0159294518807,0.25000264740765843,0.001536526072264158,0.006143456881398212,0.0003847933699806447,45.191717,10009,120.0
11,10011,1000011,60,7.1,NaN,34.0,1,0,0.811961,0.690265,...,675.9415436864738,2703.766156421032,168.98539050283424,0.2500045812157936,0.001648612550879773,0.006576125340344737,0.000416734353513532,21.483836,10011,34.0
14,10014,1000014,48,NaN,0.07,62.0,0,0,0.594735,0.430479,...,682.7224228200037,2730.887321718722,170.68119809532408,0.25002115679725584,0.0016072422595847418,0.006425442905462996,0.000402692098115178,146.764802,10014,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,11461,1001485,70,15.0,NaN,200.0,0,0,0.765609,0.637642,...,701.2069171310322,2804.817608303127,175.30424433800854,0.25003740566165333,0.001513040675927786,0.006047449734465104,0.00037943841129345633,125.060821,11461,200.0
1486,11463,1001487,71,3.3,NaN,49.0,0,0,0.807926,0.63308,...,675.1110372340426,2697.8998475330154,169.41383465929934,0.25288454924798237,0.001673155424130638,0.006575016048600207,0.0004476902680132455,16.403050,11463,49.0
1488,11465,1001489,65,6.4,NaN,50.0,2,1,0.856173,0.695205,...,687.8183063156852,2751.2490395102122,171.96062301705334,0.2500511228435455,0.0015816450617561774,0.006324324095082651,0.0003959753034245592,28.183421,11465,50.0
1493,11470,1001494,60,9.1,NaN,47.0,0,0,0.857483,0.74089,...,680.279392971246,680.279392971246,680.279392971246,1.0,0.0016159825807663263,0.0016159825807663263,0.0016159825807663263,23.860159,11470,47.0


In [121]:
print("is any null=",full_feats.isnull().values.any())
print("Count of null=",full_feats.isnull().sum().sum())

is any null= True
Count of null= 571


## Fill the missing values via Simple Imputer

In [156]:
cols_isNan =full_feats.isna().any().tolist()
for i in range(0,len(cols_isNan)):
    if cols_isNan[i]: print(full_feats.columns[i])

psa
psad
prostate_volume
prostate_volume_T2W
prostate_volume_ADC


In [184]:
full_feats_filled_simple  = full_feats.copy()

In [185]:
from sklearn.impute import SimpleImputer
cols_isNan =full_feats.isna().any().tolist()
for i in range(0,len(cols_isNan)):
    if cols_isNan[i]: 
        col_name =  full_feats.columns[i]
        print("filling col {} with SimpleImputer with mean value".format(col_name))
        simpleImp = SimpleImputer(strategy="mean") #(mean, median,most_frequent) 
        simpleImp.fit(full_feats[col_name].to_numpy().reshape(-1,1))
        full_feats_filled_simple[col_name] = simpleImp.transform(full_feats[col_name].to_numpy().reshape(-1,1))


filling col psa with SimpleImputer with mean value
filling col psad with SimpleImputer with mean value
filling col prostate_volume with SimpleImputer with mean value
filling col prostate_volume_T2W with SimpleImputer with mean value
filling col prostate_volume_ADC with SimpleImputer with mean value


In [186]:
full_feats.head()

,patient_id,study_id,patient_age,psa,psad,prostate_volume,case_ISUP,case_csPCa,original_shape_Elongation_T2W,original_shape_Flatness_T2W,...,original_gldm_LargeDependenceEmphasis_ADC,original_gldm_LargeDependenceHighGrayLevelEmphasis_ADC,original_gldm_LargeDependenceLowGrayLevelEmphasis_ADC,original_gldm_LowGrayLevelEmphasis_ADC,original_gldm_SmallDependenceEmphasis_ADC,original_gldm_SmallDependenceHighGrayLevelEmphasis_ADC,original_gldm_SmallDependenceLowGrayLevelEmphasis_ADC,ellipse_vol_ADC,patient_id_ADC,prostate_volume_ADC
0,10000,1000000,73,7.7,NaN,55.0,0,0,0.765154,0.635129,...,662.6213506858952,2582.7996834329933,182.57676749912065,0.28133518730214563,0.0017020923999421496,0.006513236251134446,0.0004993064371440754,29.154570,10000,55.0
1,10001,1000001,64,8.7,0.09,102.0,0,0,0.86035,0.754146,...,694.2544193414664,2777.0175274635794,173.56364231093806,0.2500041639523866,0.0015477373068549295,0.006189098581914566,0.0003873969880900204,125.379075,10001,102.0
2,10002,1000002,58,4.2,0.06,74.0,0,0,0.974379,0.798054,...,676.9553411313988,2466.3989444082167,229.59444031219434,0.3518427005216824,0.0016537864554952901,0.005719109538540684,0.0006374556847339416,71.760002,10002,74.0
3,10003,1000003,72,13.0,NaN,71.5,0,0,0.97144,0.795884,...,691.217092990951,2764.41773495793,172.9169324992062,0.2504651035878711,0.0015755434930430682,0.006258803746919392,0.00040472842957398715,72.334082,10003,71.5
4,10004,1000004,67,8.0,0.10,78.0,0,0,0.820187,0.63227,...,687.1358459869848,2748.5376528298166,171.78539427627686,0.2500462186945376,0.0015866646163004506,0.00633894395197835,0.0003985947823809755,83.026943,10004,78.0


In [187]:
full_feats_filled_simple.head()

,patient_id,study_id,patient_age,psa,psad,prostate_volume,case_ISUP,case_csPCa,original_shape_Elongation_T2W,original_shape_Flatness_T2W,...,original_gldm_LargeDependenceEmphasis_ADC,original_gldm_LargeDependenceHighGrayLevelEmphasis_ADC,original_gldm_LargeDependenceLowGrayLevelEmphasis_ADC,original_gldm_LowGrayLevelEmphasis_ADC,original_gldm_SmallDependenceEmphasis_ADC,original_gldm_SmallDependenceHighGrayLevelEmphasis_ADC,original_gldm_SmallDependenceLowGrayLevelEmphasis_ADC,ellipse_vol_ADC,patient_id_ADC,prostate_volume_ADC
0,10000,1000000,73,7.7,0.224438,55.0,0,0,0.765154,0.635129,...,662.6213506858952,2582.7996834329933,182.57676749912065,0.28133518730214563,0.0017020923999421496,0.006513236251134446,0.0004993064371440754,29.154570,10000,55.0
1,10001,1000001,64,8.7,0.090000,102.0,0,0,0.86035,0.754146,...,694.2544193414664,2777.0175274635794,173.56364231093806,0.2500041639523866,0.0015477373068549295,0.006189098581914566,0.0003873969880900204,125.379075,10001,102.0
2,10002,1000002,58,4.2,0.060000,74.0,0,0,0.974379,0.798054,...,676.9553411313988,2466.3989444082167,229.59444031219434,0.3518427005216824,0.0016537864554952901,0.005719109538540684,0.0006374556847339416,71.760002,10002,74.0
3,10003,1000003,72,13.0,0.224438,71.5,0,0,0.97144,0.795884,...,691.217092990951,2764.41773495793,172.9169324992062,0.2504651035878711,0.0015755434930430682,0.006258803746919392,0.00040472842957398715,72.334082,10003,71.5
4,10004,1000004,67,8.0,0.100000,78.0,0,0,0.820187,0.63227,...,687.1358459869848,2748.5376528298166,171.78539427627686,0.2500462186945376,0.0015866646163004506,0.00633894395197835,0.0003985947823809755,83.026943,10004,78.0


## Fill the missing values via Iterative Imputer

In [189]:
iterativeImp = IterativeImputer(max_iter=10, random_state=IterativeImputer_seed) #estimator=BayesianRidge
iterativeImp.fit(full_feats) #Fit the imputer on data

/u/home/ahy/.conda/envs/morph_2/lib/python3.9/site-packages/sklearn/impute/_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


IterativeImputer(random_state=0)

## should have No Null or Nan Now

In [190]:
full_feats_filled_complex_np = iterativeImp.transform(full_feats)
full_feats_filled_complex = pd.DataFrame(full_feats_filled_complex_np, columns = full_feats.columns)
print(full_feats_filled_complex.shape)
print("is any null",full_feats_filled_complex.isnull().values.any())
print("Count of null",full_feats_filled_complex.isnull().sum().sum())

(1498, 218)
is any null False
Count of null 0


In [191]:
full_feats_filled_complex[full_feats_filled_complex.isnull().any(axis=1)]

,patient_id,study_id,patient_age,psa,psad,prostate_volume,case_ISUP,case_csPCa,original_shape_Elongation_T2W,original_shape_Flatness_T2W,...,original_gldm_LargeDependenceEmphasis_ADC,original_gldm_LargeDependenceHighGrayLevelEmphasis_ADC,original_gldm_LargeDependenceLowGrayLevelEmphasis_ADC,original_gldm_LowGrayLevelEmphasis_ADC,original_gldm_SmallDependenceEmphasis_ADC,original_gldm_SmallDependenceHighGrayLevelEmphasis_ADC,original_gldm_SmallDependenceLowGrayLevelEmphasis_ADC,ellipse_vol_ADC,patient_id_ADC,prostate_volume_ADC


In [192]:
full_feats_filled_complex[full_feats_filled_complex.isna().any(axis=1)]


,patient_id,study_id,patient_age,psa,psad,prostate_volume,case_ISUP,case_csPCa,original_shape_Elongation_T2W,original_shape_Flatness_T2W,...,original_gldm_LargeDependenceEmphasis_ADC,original_gldm_LargeDependenceHighGrayLevelEmphasis_ADC,original_gldm_LargeDependenceLowGrayLevelEmphasis_ADC,original_gldm_LowGrayLevelEmphasis_ADC,original_gldm_SmallDependenceEmphasis_ADC,original_gldm_SmallDependenceHighGrayLevelEmphasis_ADC,original_gldm_SmallDependenceLowGrayLevelEmphasis_ADC,ellipse_vol_ADC,patient_id_ADC,prostate_volume_ADC


In [193]:
cols_isNan =full_feats_filled_complex.isna().any().tolist()
for i in range(0,len(cols_isNan)):
    if cols_isNan[i]: print(full_feats_filled_complex.columns[i])
    full_feats_filled_complex.isna().any()

# Visuazlize the filled value 
As you can see the difference between the full_feats (one with missing values) and the full_feats_filled (filled the missing values)

In [194]:
full_feats.head()

,patient_id,study_id,patient_age,psa,psad,prostate_volume,case_ISUP,case_csPCa,original_shape_Elongation_T2W,original_shape_Flatness_T2W,...,original_gldm_LargeDependenceEmphasis_ADC,original_gldm_LargeDependenceHighGrayLevelEmphasis_ADC,original_gldm_LargeDependenceLowGrayLevelEmphasis_ADC,original_gldm_LowGrayLevelEmphasis_ADC,original_gldm_SmallDependenceEmphasis_ADC,original_gldm_SmallDependenceHighGrayLevelEmphasis_ADC,original_gldm_SmallDependenceLowGrayLevelEmphasis_ADC,ellipse_vol_ADC,patient_id_ADC,prostate_volume_ADC
0,10000,1000000,73,7.7,NaN,55.0,0,0,0.765154,0.635129,...,662.6213506858952,2582.7996834329933,182.57676749912065,0.28133518730214563,0.0017020923999421496,0.006513236251134446,0.0004993064371440754,29.154570,10000,55.0
1,10001,1000001,64,8.7,0.09,102.0,0,0,0.86035,0.754146,...,694.2544193414664,2777.0175274635794,173.56364231093806,0.2500041639523866,0.0015477373068549295,0.006189098581914566,0.0003873969880900204,125.379075,10001,102.0
2,10002,1000002,58,4.2,0.06,74.0,0,0,0.974379,0.798054,...,676.9553411313988,2466.3989444082167,229.59444031219434,0.3518427005216824,0.0016537864554952901,0.005719109538540684,0.0006374556847339416,71.760002,10002,74.0
3,10003,1000003,72,13.0,NaN,71.5,0,0,0.97144,0.795884,...,691.217092990951,2764.41773495793,172.9169324992062,0.2504651035878711,0.0015755434930430682,0.006258803746919392,0.00040472842957398715,72.334082,10003,71.5
4,10004,1000004,67,8.0,0.10,78.0,0,0,0.820187,0.63227,...,687.1358459869848,2748.5376528298166,171.78539427627686,0.2500462186945376,0.0015866646163004506,0.00633894395197835,0.0003985947823809755,83.026943,10004,78.0


In [195]:
full_feats_filled_complex.head()

,patient_id,study_id,patient_age,psa,psad,prostate_volume,case_ISUP,case_csPCa,original_shape_Elongation_T2W,original_shape_Flatness_T2W,...,original_gldm_LargeDependenceEmphasis_ADC,original_gldm_LargeDependenceHighGrayLevelEmphasis_ADC,original_gldm_LargeDependenceLowGrayLevelEmphasis_ADC,original_gldm_LowGrayLevelEmphasis_ADC,original_gldm_SmallDependenceEmphasis_ADC,original_gldm_SmallDependenceHighGrayLevelEmphasis_ADC,original_gldm_SmallDependenceLowGrayLevelEmphasis_ADC,ellipse_vol_ADC,patient_id_ADC,prostate_volume_ADC
0,10000.0,1000000.0,73.0,7.7,0.240268,55.0,0.0,0.0,0.765154,0.635129,...,662.621351,2582.799683,182.576767,0.281335,0.001702,0.006513,0.000499,29.154570,10000.0,55.0
1,10001.0,1000001.0,64.0,8.7,0.090000,102.0,0.0,0.0,0.860350,0.754146,...,694.254419,2777.017527,173.563642,0.250004,0.001548,0.006189,0.000387,125.379075,10001.0,102.0
2,10002.0,1000002.0,58.0,4.2,0.060000,74.0,0.0,0.0,0.974379,0.798054,...,676.955341,2466.398944,229.594440,0.351843,0.001654,0.005719,0.000637,71.760002,10002.0,74.0
3,10003.0,1000003.0,72.0,13.0,0.240268,71.5,0.0,0.0,0.971440,0.795884,...,691.217093,2764.417735,172.916932,0.250465,0.001576,0.006259,0.000405,72.334082,10003.0,71.5
4,10004.0,1000004.0,67.0,8.0,0.100000,78.0,0.0,0.0,0.820187,0.632270,...,687.135846,2748.537653,171.785394,0.250046,0.001587,0.006339,0.000399,83.026943,10004.0,78.0


In [196]:
full_feats_filled_simple.head()

,patient_id,study_id,patient_age,psa,psad,prostate_volume,case_ISUP,case_csPCa,original_shape_Elongation_T2W,original_shape_Flatness_T2W,...,original_gldm_LargeDependenceEmphasis_ADC,original_gldm_LargeDependenceHighGrayLevelEmphasis_ADC,original_gldm_LargeDependenceLowGrayLevelEmphasis_ADC,original_gldm_LowGrayLevelEmphasis_ADC,original_gldm_SmallDependenceEmphasis_ADC,original_gldm_SmallDependenceHighGrayLevelEmphasis_ADC,original_gldm_SmallDependenceLowGrayLevelEmphasis_ADC,ellipse_vol_ADC,patient_id_ADC,prostate_volume_ADC
0,10000,1000000,73,7.7,0.224438,55.0,0,0,0.765154,0.635129,...,662.6213506858952,2582.7996834329933,182.57676749912065,0.28133518730214563,0.0017020923999421496,0.006513236251134446,0.0004993064371440754,29.154570,10000,55.0
1,10001,1000001,64,8.7,0.090000,102.0,0,0,0.86035,0.754146,...,694.2544193414664,2777.0175274635794,173.56364231093806,0.2500041639523866,0.0015477373068549295,0.006189098581914566,0.0003873969880900204,125.379075,10001,102.0
2,10002,1000002,58,4.2,0.060000,74.0,0,0,0.974379,0.798054,...,676.9553411313988,2466.3989444082167,229.59444031219434,0.3518427005216824,0.0016537864554952901,0.005719109538540684,0.0006374556847339416,71.760002,10002,74.0
3,10003,1000003,72,13.0,0.224438,71.5,0,0,0.97144,0.795884,...,691.217092990951,2764.41773495793,172.9169324992062,0.2504651035878711,0.0015755434930430682,0.006258803746919392,0.00040472842957398715,72.334082,10003,71.5
4,10004,1000004,67,8.0,0.100000,78.0,0,0,0.820187,0.63227,...,687.1358459869848,2748.5376528298166,171.78539427627686,0.2500462186945376,0.0015866646163004506,0.00633894395197835,0.0003985947823809755,83.026943,10004,78.0


## Normalize the features

### all features

In [197]:
full_feats_filled_simple_normalized=(full_feats_filled_simple-full_feats_filled_simple.mean())/full_feats_filled_simple.std()
full_feats_filled_simple_normalized["case_csPCa"]=full_feats_filled_simple["case_csPCa"].astype('int')
full_feats_filled_simple_normalized["patient_id"]=full_feats_filled_simple["patient_id"].astype('int')
full_feats_filled_simple_normalized["study_id"]=full_feats_filled_simple["study_id"].astype('int')

In [198]:
full_feats_filled_complex_normalized=(full_feats_filled_complex-full_feats_filled_complex.mean())/full_feats_filled_complex.std()
full_feats_filled_complex_normalized["case_csPCa"]=full_feats_filled_complex["case_csPCa"].astype('int')
full_feats_filled_complex_normalized["patient_id"]=full_feats_filled_complex["patient_id"].astype('int')
full_feats_filled_complex_normalized["study_id"]=full_feats_filled_complex["study_id"].astype('int')

### adc and Tw2 features

In [199]:
t2w_adc_feats_normalized=(t2w_adc_feats-t2w_adc_feats.mean())/t2w_adc_feats.std()
t2w_adc_feats_normalized["patient_id_T2W"]=t2w_adc_feats["patient_id_T2W"].astype('int')
t2w_adc_feats_normalized["patient_id_ADC"]=t2w_adc_feats["patient_id_ADC"].astype('int')

## Visualize the normalized values vs not

In [200]:
full_feats_filled_simple.head()

,patient_id,study_id,patient_age,psa,psad,prostate_volume,case_ISUP,case_csPCa,original_shape_Elongation_T2W,original_shape_Flatness_T2W,...,original_gldm_LargeDependenceEmphasis_ADC,original_gldm_LargeDependenceHighGrayLevelEmphasis_ADC,original_gldm_LargeDependenceLowGrayLevelEmphasis_ADC,original_gldm_LowGrayLevelEmphasis_ADC,original_gldm_SmallDependenceEmphasis_ADC,original_gldm_SmallDependenceHighGrayLevelEmphasis_ADC,original_gldm_SmallDependenceLowGrayLevelEmphasis_ADC,ellipse_vol_ADC,patient_id_ADC,prostate_volume_ADC
0,10000,1000000,73,7.7,0.224438,55.0,0,0,0.765154,0.635129,...,662.6213506858952,2582.7996834329933,182.57676749912065,0.28133518730214563,0.0017020923999421496,0.006513236251134446,0.0004993064371440754,29.154570,10000,55.0
1,10001,1000001,64,8.7,0.090000,102.0,0,0,0.86035,0.754146,...,694.2544193414664,2777.0175274635794,173.56364231093806,0.2500041639523866,0.0015477373068549295,0.006189098581914566,0.0003873969880900204,125.379075,10001,102.0
2,10002,1000002,58,4.2,0.060000,74.0,0,0,0.974379,0.798054,...,676.9553411313988,2466.3989444082167,229.59444031219434,0.3518427005216824,0.0016537864554952901,0.005719109538540684,0.0006374556847339416,71.760002,10002,74.0
3,10003,1000003,72,13.0,0.224438,71.5,0,0,0.97144,0.795884,...,691.217092990951,2764.41773495793,172.9169324992062,0.2504651035878711,0.0015755434930430682,0.006258803746919392,0.00040472842957398715,72.334082,10003,71.5
4,10004,1000004,67,8.0,0.100000,78.0,0,0,0.820187,0.63227,...,687.1358459869848,2748.5376528298166,171.78539427627686,0.2500462186945376,0.0015866646163004506,0.00633894395197835,0.0003985947823809755,83.026943,10004,78.0


In [201]:
full_feats_filled_simple_normalized.head()

,patient_id,study_id,patient_age,psa,psad,prostate_volume,case_ISUP,case_csPCa,original_shape_Elongation_T2W,original_shape_Flatness_T2W,...,original_gldm_LargeDependenceEmphasis_ADC,original_gldm_LargeDependenceHighGrayLevelEmphasis_ADC,original_gldm_LargeDependenceLowGrayLevelEmphasis_ADC,original_gldm_LowGrayLevelEmphasis_ADC,original_gldm_SmallDependenceEmphasis_ADC,original_gldm_SmallDependenceHighGrayLevelEmphasis_ADC,original_gldm_SmallDependenceLowGrayLevelEmphasis_ADC,ellipse_vol_ADC,patient_id_ADC,prostate_volume_ADC
0,10000,1000000,1.029209,-0.287234,2.461960e-16,-0.279400,-0.705929,0,-1.175607,-1.10159,...,-2.051181,0.430926,-0.544297,-0.506616,1.417104,0.643569,-0.4492,-0.699413,-1.728815,-0.279400
1,10001,1000001,-0.222329,-0.219422,-3.974937e-01,1.021754,-0.705929,0,-0.015216,0.550774,...,1.174161,0.648656,-0.584759,-0.602858,-1.189592,0.488914,-0.665584,1.432062,-1.726463,1.021754
2,10002,1000002,-1.056687,-0.524577,-4.861952e-01,0.246598,-0.705929,0,1.374727,1.160354,...,-0.589672,0.300434,-0.333223,-0.290034,0.60133,0.264669,-0.18208,0.244342,-1.724110,0.246598
3,10003,1000003,0.890150,0.072170,2.461960e-16,0.177388,-0.705929,0,1.338907,1.130232,...,0.864472,0.634531,-0.587662,-0.601442,-0.720011,0.522172,-0.632072,0.257059,-1.721758,0.177388
4,10004,1000004,0.194851,-0.266891,-3.679266e-01,0.357335,-0.705929,0,-0.504788,-1.141289,...,0.448343,0.616729,-0.592742,-0.602729,-0.532201,0.560409,-0.643932,0.493917,-1.719406,0.357335


In [ ]:
full_feats_filled_complex.head()

In [202]:
full_feats_filled_complex_normalized.head()

,patient_id,study_id,patient_age,psa,psad,prostate_volume,case_ISUP,case_csPCa,original_shape_Elongation_T2W,original_shape_Flatness_T2W,...,original_gldm_LargeDependenceEmphasis_ADC,original_gldm_LargeDependenceHighGrayLevelEmphasis_ADC,original_gldm_LargeDependenceLowGrayLevelEmphasis_ADC,original_gldm_LowGrayLevelEmphasis_ADC,original_gldm_SmallDependenceEmphasis_ADC,original_gldm_SmallDependenceHighGrayLevelEmphasis_ADC,original_gldm_SmallDependenceLowGrayLevelEmphasis_ADC,ellipse_vol_ADC,patient_id_ADC,prostate_volume_ADC
0,10000,1000000,1.029209,0.045512,-0.035969,-0.031912,-0.705929,0,-1.175607,-1.101590,...,-2.051181,0.430926,-0.544297,-0.506616,1.417104,0.643569,-0.449200,-0.699413,-1.728815,0.031912
1,10001,1000001,-0.222329,0.047155,-0.312474,-0.031912,-0.705929,0,-0.015216,0.550774,...,1.174161,0.648656,-0.584759,-0.602858,-1.189592,0.488914,-0.665584,1.432062,-1.726463,0.031912
2,10002,1000002,-1.056687,0.039760,-0.367676,-0.031912,-0.705929,0,1.374727,1.160354,...,-0.589672,0.300434,-0.333223,-0.290034,0.601330,0.264669,-0.182080,0.244342,-1.724110,0.031912
3,10003,1000003,0.890150,0.054221,-0.035969,-0.031912,-0.705929,0,1.338907,1.130232,...,0.864472,0.634531,-0.587662,-0.601442,-0.720011,0.522172,-0.632072,0.257059,-1.721758,0.031912
4,10004,1000004,0.194851,0.046005,-0.294073,-0.031912,-0.705929,0,-0.504788,-1.141289,...,0.448343,0.616729,-0.592742,-0.602729,-0.532201,0.560409,-0.643932,0.493917,-1.719406,0.031912


## Save dataframe

In [203]:
full_feats_filled_simple_normalized.to_csv("/data1/practical-sose23/morphometric/picai_labels_cropped/clinical_information/marksheet_filled_simple_normalized.csv")
print("/data1/practical-sose23/morphometric/picai_labels_cropped/clinical_information/marksheet_filled_simple_normalized.csv")

full_feats_filled_complex_normalized.to_csv("/data1/practical-sose23/morphometric/picai_labels_cropped/clinical_information/marksheet_filled_complex_normalized.csv")
print("/data1/practical-sose23/morphometric/picai_labels_cropped/clinical_information/marksheet_filled_complex_normalized.csv")

t2w_adc_feats_normalized.to_pickle("/data1/practical-sose23/morphometric/t2w_adc_feats_normalized.pkl")
print("/data1/practical-sose23/morphometric/t2w_adc_feats_normalized.pkl")

/data1/practical-sose23/morphometric/picai_labels_cropped/clinical_information/marksheet_filled_simple_normalized.csv
/data1/practical-sose23/morphometric/picai_labels_cropped/clinical_information/marksheet_filled_complex_normalized.csv
/data1/practical-sose23/morphometric/t2w_adc_feats_normalized.pkl
